###1.Выбрать себе датасет (который интересен или нравится больше всего, можно глянуть здесь https://economic-caper-a4c.notion.site/d062c410b90145bca90fc23b1348c813), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.


####ссылка https://www.kaggle.com/datasets/imoore/60k-stack-overflow-questions-with-quality-rate

###Это набор данных, содержащий 60 000 вопросов о переполнении стека за 2016-2020 годы. Вопросы классифицированы по трем категориям в зависимости от их качества:

#####HQ: Высококачественные посты без единой правки.
#####LQ_EDIT: посты низкого качества с отрицательной оценкой и многочисленные правки сообщества. Однако они по-прежнему остаются открытыми после этих изменений.
#####LQ_CLOSE: сообщения низкого качества, которые были закрыты сообществом без единой правки.

###Данные разбиты на 2 датасета. Используем  датасет train, чтобы настроить параметры модели таким образом, чтобы она могла точно предсказывать целевую переменную на данных датасета valid.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [2]:
df = pd.read_csv("./train.csv")
df.head(3)

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            45000 non-null  int64 
 1   Title         45000 non-null  object
 2   Body          45000 non-null  object
 3   Tags          45000 non-null  object
 4   CreationDate  45000 non-null  object
 5   Y             45000 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [4]:
df['Y'].value_counts()

LQ_CLOSE    15000
HQ          15000
LQ_EDIT     15000
Name: Y, dtype: int64

###Преобразуем целевую переменную к бинарному виду. Примем посты высокого качества за "0", а низкого качества - "1".

In [5]:
df['Y'] = df['Y'].apply(lambda x: 0 if x == 'HQ' else 1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Y'],
                                                    test_size=0.25, random_state=42)

X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)


X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [7]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    

class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [8]:
features = ['Title', 'Body']
target = 'Y'

In [9]:
title_ = Pipeline([
                ('imputer', TextImputer('Title', '')),
                ('selector', ColumnSelector(key='Title')),
                ('tfidf', TfidfVectorizer())
            ])

body_ = Pipeline([
                ('imputer', TextImputer('Body', '')),
                ('selector', ColumnSelector(key='Body')),
                ('tfidf', TfidfVectorizer())
            ])

feats = FeatureUnion([('Title', title_),
                      ('Body', body_)])

In [10]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 11.6 s, sys: 8.04 s, total: 19.6 s
Wall time: 14.2 s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('Body',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='Body',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='Body')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [11]:
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import dill

In [13]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

In [14]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [15]:
X_test.head(3)

,Id,Title,Body,Tags,CreationDate,Y
0,54233203,not able to run function in php,<p>I'am not able to run a function in php oops...,<php><oop>,2019-01-17 09:54:10,1
1,39253307,Export GIT LOG into an Excel file,"<p>I have looked into the forum, but with no l...",<git><logging>,2016-08-31 15:13:57,0
2,58793066,How to summarize the employees by net revenue ...,8.\tThe sales director would like to reward th...,<sql>,2019-11-10 21:25:11,1


In [16]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [17]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [18]:
preds[:5]

array([0.99046622, 0.62612662, 0.95233769, 0.91124358, 0.94457124])

In [19]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.504651816489683, F-Score=0.926, Precision=0.919, Recall=0.932


In [20]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [22]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2P9h2WbU6p56dy0BzS3npFrYe9J_7axXJ3UCPrKZtxAd5AaH8

--2023-05-01 18:25:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz.1’

ngrok-stable-linux- 100%[===================>]  13.21M  19.1MB/s    in 0.7s    

2023-05-01 18:25:29 (19.1 MB/s) - ‘ngrok-stable-linux-amd64.tgz.1’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [23]:
!./ngrok http 8000 -log=stdout &

INFO[05-01|18:25:30] no configuration paths supplied 
INFO[05-01|18:25:30] using configuration at default config path path=/root/.ngrok2/ngrok.yml
INFO[05-01|18:25:30] open config file                         path=/root/.ngrok2/ngrok.yml err=nil
t=2023-05-01T18:25:30+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
t=2023-05-01T18:25:30+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2023-05-01T18:25:30+0000 lvl=info msg="client session established" obj=csess id=99d8ca7a707f
t=2023-05-01T18:25:30+0000 lvl=info msg="started tunnel" obj=tunnels name="command_line (http)" addr=http://localhost:8000 url=http://40e2-35-233-158-247.ngrok-free.app
t=2023-05-01T18:25:30+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=http://localhost:8000 url=https://40e2-35-233-158-247.ngrok-free.app
t=2023-05-01T18:25:48+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
t=2023-05-01T18:25:48+0000 lvl=info msg="sess

In [24]:
# Загружаем обученные модели
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [25]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [26]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    title_, body_ = "", ""
    request_json = request.get_json()
    
    if request_json["Title"]:
        title_ = request_json['Title']
    
    if request_json["Body"]:
        body_ = request_json['Body']
                
        
    print(title_)  
    preds = model.predict_proba(pd.DataFrame({"Title": [title_],
                                              "Body": [body_],
                                              }))
    data["predictions"] = preds[:, 1][0]
    data["Title"] = title_
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://7c02-35-233-158-247.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/May/2023 18:27:27] "POST /predict HTTP/1.1" 200 -


Export GIT LOG into an Excel file
OK


...

argument of type'NoneType' is not itrable
OK


INFO:werkzeug:127.0.0.1 - - [01/May/2023 18:27:37] "POST /predict HTTP/1.1" 200 -


I am getting a StringIndexOutOfBoundsException when I attempt to use array parts from the children classes
OK


INFO:werkzeug:127.0.0.1 - - [01/May/2023 18:27:37] "POST /predict HTTP/1.1" 200 -


In python-telegram-bot how to get all participants of the group?
OK


###Проверим 

In [27]:
import requests

import urllib.request
import json 

In [28]:
title_, body_ = ( 
    "Export GIT LOG into an Excel file",
    "I have looked into the forum, but with"
)

body = {
        'Title': title_, 
        'Body': body_
        }

In [29]:
with app.test_client() as t:
    response = t.post('/predict', json=body)
    json_data = response.get_json()

json_data

Export GIT LOG into an Excel file
OK


{'Title': 'Export GIT LOG into an Excel file',
 'predictions': 0.689907026637619,
 'success': True}